<a href="https://colab.research.google.com/github/youngchanseo/Hon.Gong.Machine-SNU/blob/main/(05_2_3)_%ED%95%98%EC%9D%B4%ED%8D%BC%ED%8C%8C%EB%9D%BC%EB%AF%B8%ED%84%B0_%ED%8A%9C%EB%8B%9D_p248.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
#Full Code: https://bit.ly/hg-05-2

import pandas as pd
wine = pd.read_csv('https://raw.githubusercontent.com/youngchanseo/Hon.Gong.Machine-SNU/main/(05_2_0)_Raw%20data.csv')
#wine = pd.read_csv('htpps://bit.ly/wine_csv_data')

data = wine [['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

#훈련 세트의 입력 데이터와 target data를 train-input과 train_target 배열에 저장
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

# 모델을 만듦
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

# cross validatee ()이라는 교차 검증함수: 기본은 5폴드 교차 검증
# fit_time (모델을 훈련하는 시간), Score_time(모델을 검증하는 시간), test_score 키를 가진 딕셔너리 반환
from sklearn.model_selection import cross_validate
# scores = cross_validate(dt, train_input, train_target)

#타깃 클래스를 골고루 나누기 위해 StratifiedkFold 를 사용
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())

# 10폴드 교차 검증
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)

#하이퍼 파라미터
모델 파라미터
- 모신러닝 모델이 학습하는 파라미터
하이퍼파라미터
- 사용자가 지정해야만 하는 파라미터
- 사이킷런과 같은 머신러닝 라이브러리 사용: 클래스나 매서드의 매개변수로 표현

  (1) 라이브러리가 제공하는 기본값 그대로 사용
  (2) 검증세트의 점수나 교착 검증을 통해 매개변수를 조금씩 바꿔봄. 모델마다 적게는 1 ~ 2개에서 5 ~ 6개의 매개변수를 제공함
  (3) 매개변수를 바꿔가면서 모델훈련 & 교차검증 수행

사이킷 런의 Grid searchCV class
- 하이퍼 파라미터 탐색과 교자 검증을 한 번에 수행
- -별도로 cross_validate() 수행할 필요 없음

In [61]:
# GridSearchCV class 에 탐색 대상 모델과 params 변수를 전달하여 grid search 객체를 만듦
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]} # 0.0001부터 0.0005까지 0.0001씩 증가하는 5개의 값
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [62]:
#Grid Search (gs) 객체에 fit() method 호출: greed search 객체는 결정 트리 모델 min_inpurity_decreaase 값을 바꿔가며 총 5번 실행
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

GreadSearchCV의 cv 매개변수 기본값은 5.

따라서min_impurity_decrease 값마다 5- 폴드 교차 검증을 수행함. 결국 5*5 =25개의 모델을 훈련

In [63]:
# gs객체의 best_estimator_속성:
#Grid search 훈련이 끝나면 25개의 모델중에서 검증 점수가 가장 높은 모델의  매개변수 조합으로 전체 훈련 세트에서 자동으로 다시 모델 훈련
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [64]:
#Grid Search로 찾은 최적의 매개변수는 best_params_ 속성에 저장되어 있음
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


여기에서는 0.0001이 가장 좋은 값으로 선택됨

각 매개변수에서 수행한 교차 검증의 평균 점수는 cv_results_속서으이 'mean_test_score'키에 저장되어 있음

In [65]:
# 5번의 교차 검증으로 얻은 점수 출력

print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


첫번째 값이 가장 큼

수동으로 고르는것보다 argmax() 함수를 사용하면 가장 큰 값의 인덱스 추출 가능
 그 다음 인덱스를 사용해 params 키에 저장된 매개변수를 출력

In [66]:
#최상의 검증 점수를 만든 매개변수 조합

import numpy as np
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


과정 정리

1) 먼저 탐색할 매개변수 저장

2) 훈련 세트에서 grid search를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾고, grid search 객체에 저장

3)grid search는 최상의 매개변수에서 (교차 검증ㅇ 사용한 훈련 세트가 아니라) 전체 훈련 세트를 사용해 최종 모델 훈련 > 이 모델도 grid search 객체에 저장됨

In [67]:
# Decision Tree에서 min_impurity_decrease는 노드를 분할하기 위한 불순도 감소 최소량을 지정함.
# max_depth로 트리 깊이 제한하고, min_samples_ split으로 노드를 나누기위한 최소 샘플수 고름
params = { # arrange(): 첫 번째 매개변수 값(0.0001) 에서 시작하여 2번째 매개변수 (0.001)에 도달할때까지 세번째 매개변수 (0.0001) 를 계속 더함
          'min_impurity_decrease': np.arange (0.0001, 0.001, 0.0001),
           'max_depth': range(5, 20, 1), #max_depth를 5에서 20까지 1씩 증가하면서 15개의 값을 만듦
           'min_samples_split': range(2, 100, 10) #min_samples_split: 2에서 100까지 10씩 증가하면서 10개의 값을 만듦
          }
          # 따라서 이 매개변수로 수행할 교차 검증 횟수는 9* 15*10, = 1,350개
          # 기존 5 폴드 교차 검증 수행하여 만들어지는 모델의 수는 6,750개

In [68]:
# n_jobs 매개변수를 -1로 설정하고 grid search를 실행
gs = GridSearchCV (DecisionTreeClassifier (random_state = 42), params, n_jobs = -1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [69]:
#최상의 매개변수 조합 확인
print (gs.best_params_)

#최상의 교차 검증 점수 확인
print(np.max(gs.cv_results_['mean_test_score']))

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}
0.8683865773302731


GridSearchCV 클래스
- 원하는 매개변수 값을 나열하면 자동으로 교차 검증을 수행 => 최상의 매개변수를 찾을 수 있음

# 랜덤 서치 Random Search

매개변수의 값이 수치일 때값의 범위나 간격을 미리 정하기 어려울 수 있음

너무 많은 매개변수 조건이 있어 Grid Search 수행 시간이 오래걸릴 수 있음

Rnadom Search
- 매개변수를 샘플링하는 확률 분포 객체를 전달
-  scipy의 stats sub package에 있는 uniform과 randint class는 모두 주어진 범위에서 고르게 갑을 뽑음:
  - '균등 분포에서 샘플링'

In [70]:
#randint는 정수값을 뽑고 uniform은 실수값을 뽑음
from scipy.stats import uniform, randint

In [93]:
#0에서 10사이의 범위를 갖는 randint 객체를 만들고 10개 숫자를 샘플링
rgen = randint(0, 10)
rgen.rvs(10)

array([7, 2, 6, 8, 1, 1, 3, 4, 9, 2])

10개 밖에 되지 않기에, 고르게 샘플링 되지 않는 것 같음 = > 1,000개로 샘플링

In [94]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 97, 121, 105,  99, 103, 107, 100,  92,  83,  93]))

개수 늘어남: 0에서 9까지 숫자가 어느 정도 고르게 추출됨

uniform class사용법도 동일함

In [84]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.19810438, 0.18671033, 0.12346017, 0.92205913, 0.98687556,
       0.74141259, 0.21286397, 0.80248841, 0.0942301 , 0.80380938])

탐색할 매개변수의 딕셔너리
- min_samples_leaf 매개변수 (leaf node가 되기 위한 최소 샘플의 개수) 를 탐색 대상에 추가: 어떤 노드가 분할하여 만들어질 자식 노드의 샘플 수가 이 값보다 작을 경우 분할하지 않음

In [101]:
# 탐색할 매개변수의 범위
params = {'min_impurity_decrease': uniform(0.0001, 0.001), #0.0001에서 0.001사이의 실수값을 샘플링
          'max_depth': randint(20, 50), #20에서 50까지의 정수를 샘플링
          'min_samples_split': randint(2, 25),# 1에서 25까지의 정수를 샘플
          'min_samples_leaf': randint(1, 25),
          }
          # 샘플링 횟수: 사이킷런의 random search class인 RandomizedSearchCV의 n_iter 매개변수에 저장



In [104]:
# 100번 (n_iter 매개변수)를 샘플링하여 교차 검증을 수행하고,최적의 매개변수조합을 찾음
# 앞서 Grid Search 보다 훨씬 교차 검증 수를 줄이면서 넓은 영역을 효과적으로 탐색가능
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)



RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7b9cb8889990>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7b9cb888b2b0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7b9cb890cbe0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7b9cb890d720>},
                   random_state=42)

In [105]:
#최적의 매개변수 조합 출력
print(gs.best_params_) #최적의 모델은 이미 전체 훈련 세트 (train_input, train_target)으로 훈련되어 best_estimator_속성에 저장되어 있음

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [106]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [107]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86


Test set 점수는 검증 set에 대한 점수보다 조금 작은 것이 일반적

